Install relevant packages:
- numpy (1.26.4)
- scipy (1.13.1)

In [1]:
import numpy as np
from scipy.optimize import linprog

import pickle

In [ ]:
files = ['python_1.4tolfac_1e-10eps.pkl',
         'python_2.5tolfac_8e-4eps.pkl',
         'python_1.4tolfac_1e-10eps.pkl',
         'python_2.5tolfac_8e-4eps.pkl']

for file in files:
    with open(file, 'rb') as f:
        data = pickle.load(f)
        results = data['r_array']

        diffs = [np.sum(np.abs(results[0][0][0] - results[i][j][0])) for i in range(256) for j in range(256)]
        print(max(diffs), min(diffs), sum(diffs))

        diffs = [np.sum(np.abs(results[0][0][1] - results[i][j][1])) for i in range(256) for j in range(256)]
        print(max(diffs), min(diffs), sum(diffs))

        diffs = [np.sum(np.abs(results[0][0][2] - results[i][j][2])) for i in range(256) for j in range(256)]
        print(max(diffs), min(diffs), sum(diffs))